In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices)

MDDE = "GPU" if "GPU" in [k.device_type for k in device_lib.list_local_devices()] else "CPU"
print(MDDE)

<function list_local_devices at 0x7fa755a5aed0>
CPU


In [15]:
import tensorflow as tf
import tensorflow.contrib.keras as keras

In [3]:
from __future__ import print_function

import json
from deepmoji.model_def import deepmoji_transfer
from deepmoji.global_variables import PRETRAINED_PATH
from deepmoji.finetuning import (
    load_benchmark,
    finetune)


DATASET_PATH = 'data/kaggle-insults/raw.pickle'
nb_classes = 2

with open('model/vocabulary.json', 'r') as f:
    vocab = json.load(f)
print("lets go")
# Load dataset. Extend the existing vocabulary with up to 10000 tokens from
# the training dataset.
print("loading benchmark")
data = load_benchmark(DATASET_PATH, vocab, extend_with=10000)
print("finished")
# Set up model and finetune. Note that we have to extend the embedding layer
# with the number of tokens added to the vocabulary.
model = deepmoji_transfer(nb_classes, data['maxlen'], PRETRAINED_PATH,
                          extend_embedding=data['added'])
model.summary()
model, acc = finetune(model, data['texts'], data['labels'], nb_classes,
                      data['batch_size'], method='chain-thaw')
print('Acc: {}'.format(acc))

Using TensorFlow backend.


lets go
loading benchmark
finished
Loading weights for embedding
Extended vocabulary for embedding layer from 50000 to 53545 tokens.
Loading weights for bi_lstm_0
Loading weights for bi_lstm_1
Loading weights for attlayer
Ignoring weights for softmax
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 256)      13707520    input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 50, 256)      0           embedding[0][0]                  
________________________________________________________

In [21]:
model.summary()

model.save("deepmoji_kaggle_insult.tf")
config_deepmoji_kaggle_insults = model.get_config()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 256)      13707520    input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 50, 256)      0           embedding[0][0]                  
__________________________________________________________________________________________________
embed_drop (SpatialDropout1D)   (None, 50, 256)      0           activation_1[0][0]               
__________________________________________________________________________________________________
bi_lstm_0 

In [20]:
reconstructed_model = keras.models.load_model("deepmoji_kaggle_insult.tf")

TypeError: Layer embed_drop does not support masking, but was passed an input_mask: Tensor("embedding_7/NotEqual:0", shape=(?, 50), dtype=bool)